In [1]:
from neo4j import GraphDatabase

In [2]:
URI = "neo4j://neo4j"
driver = GraphDatabase.driver(URI)

In [3]:
def read_tx(tx, query):
    return [record.values() for record in tx.run(query)]

In [4]:
def read(query):
    with driver.session(database="neo4j") as session:
        return session.execute_read(read_tx, query)

In [5]:
res = read("MATCH (n:Product) RETURN n LIMIT 3")
for r in res:
    print(r)

[<Node element_id='4:d2f580c4-f8dd-4138-8177-a677d7d52058:0' labels=frozenset({'Product'}) properties={'price': 2999.0, 'name': 'Men Slim Fit Self Design Casual Shirt', 'pid': 'SHTFRTGVPM9D4PZE', 'brand': 'Lev', 'url': 'https://www.flipkart.com/levis-men-self-design-casual-black-shirt/p/itm61441176d6218?pid=SHTFRTGVPM9D4PZE&lid=LSTSHTFRTGVPM9D4PZEYLJ5X3&marketplace=FLIPKART&srno=b_2_52&otracker=browse&fm=organic&iid=7db20866-1a4d-4a2a-b693-aac8b0a3aa53.SHTFRTGVPM9D4PZE.SEARCH&ssid=7z8q0njtsg0000001612111731089', 'desc': ''}>]
[<Node element_id='4:d2f580c4-f8dd-4138-8177-a677d7d52058:1' labels=frozenset({'Product'}) properties={'price': 2799.0, 'avg_rating': 2.0, 'name': 'Men Slim Fit Checkered Spread Collar Casual Shirt', 'pid': 'SHTFVNHSHWCCZ7YQ', 'brand': 'Lev', 'url': 'https://www.flipkart.com/levis-men-checkered-casual-blue-shirt/p/itm3a48cdaf1d157?pid=SHTFVNHSHWCCZ7YQ&lid=LSTSHTFVNHSHWCCZ7YQOMU9DD&marketplace=FLIPKART&srno=b_2_53&otracker=browse&fm=organic&iid=7db20866-1a4d-4a2a-b

In [6]:
res = read(
    """
    MATCH (sc)-[r:SUBCATEGORY_OF]->(c)
    RETURN c.name AS Category, sc.name as SubCategory
    LIMIT 3
""",
)
for r in res:
    print(r)

['Clothing and Accessories', 'Fabrics']
['Clothing and Accessories', 'Inspire Clothing and Accessories']
['Clothing and Accessories', 'INSPIRE Clothing and Accessories']


In [7]:
res = read("MATCH (n:Seller) RETURN n.name LIMIT 3")
for r in res:
    print(r)

['AestheticAroma']
['WildHunte']
['Business of Govinda Biswal(New Sell']


In [8]:
res = read("MATCH ()-[r:SELLS]->(p) WHERE r.out_of_stock RETURN p.pid LIMIT 3")
for r in res:
    print(r)

['SHOFC585JH9RGXWG']
['SHOFRF5982GGBPQZ']
['SHOE8RHSHBGZHFAR']


In [9]:
res = read(
    """
    MATCH ()-[r:SELLS]->()
    RETURN min(r.selling_price)
    """,
)
for r in res:
    print(r)

[99.0]


In [10]:
res = read(
    """
    MATCH ()-[r:SELLS]->(p)
    WHERE r.selling_price > 50 AND r.selling_price < 150
    RETURN count(1)
    """,
)
for r in res:
    print(r)

[64]


In [11]:
res = read(
    """
    MATCH ()-[r:SELLS]->(p)
    WHERE r.selling_price > 50 AND r.selling_price < 150
    RETURN p.pid LIMIT 3
    """,
)
for r in res:
    print(r)

['SOCEUWH7KVGHVSFJ']
['SOCFFGA2FYZQBFXT']
['VESFRGGT2YVHFZUA']


In [12]:
res = read(
    """
    MATCH (p:Product)
    RETURN DISTINCT p.brand
    LIMIT 3
    """,
)
for r in res:
    print(r)

['Lev']
['US']
['LA SMI']


In [13]:
res = read(
    """
    MATCH
        (p:Product)-[r1:HAS_CATEGORY]->
        (sc:SubCategory)-[r2:SUBCATEGORY_OF]->(c:Category)
    WHERE c.name = 'Toys'
    RETURN p.pid LIMIT 3
    """,
)
for r in res:
    print(r)

['HATEZMCYMGPRQWVK']


In [14]:
res = read("MATCH ()-[r:SELLS]->(p) WHERE r.discount IS NOT NULL RETURN p.pid LIMIT 3")
for r in res:
    print(r)

['SHOFN3EJHXHGSGUC']
['SHOF34H3GMHM9VNB']
['SHOFN4GRP949XZ5H']


In [15]:
res = read(
    """
    MATCH ()-[r:SELLS]->(p)
    WHERE r.discount > 80
    RETURN p.pid, r.discount ORDER BY r.discount DESC
    LIMIT 3
    """,
)
for r in res:
    print(r)

['TSHF5FRXKGF6A4FH', 87.0]
['TSHFMFXGFJ7G2ABK', 86.0]
['TSHFMFT7VASAHBH3', 86.0]


In [16]:
res = read(
    """
    MATCH (p:Product)
    RETURN DISTINCT
        SUBSTRING(
            REDUCE(x = '', y IN SPLIT(p.url, '/')[0..3] | x + '/' + y),
            1
        )
    LIMIT 3
    """,
)
for r in res:
    print(r)

['https://www.flipkart.com']
